In [34]:
import numpy as np
from midi2audio import FluidSynth
from IPython.display import Audio
from mido import MidiFile, MidiTrack

from midi2audio import FluidSynth
from IPython.display import Audio


def play_midi(midi_filename):
    FluidSynth("soundfont.sf2").midi_to_audio(midi_filename, 'test.wav')
    return Audio(filename="test.wav") 

# Update your play_midi function
def play_midi(midi_filename):
    FluidSynth("soundfont.sf2").midi_to_audio(midi_filename, 'test.wav')
    return Audio("test.wav")
def get_midi_file_notes(filename):
    """Returns the sequence of notes played in the midi file
    There are 128 possible notes on a MIDI device, and they are numbered 0 to 127.
    The middle C is note number 60. Larger numbers indiciate higher pitch notes,
    and lower numbers indicate lower pitch notes.
    """
    notes = []
    for msg in  MidiFile(filename):
        if msg.type == 'note_on':
            notes.append(msg.note)
    return notes

def gen_input_output(notes, context_length=20):
    """
    Generate a list of training data points, each of the form (x, t),
    where "x" is a list of length `context_length` consisting of the
    previous notes, and "t" is the corresponding next note.

    Parameters:
        `notes` - a sequence of notes in a piece, generated
                  from calling `get_midi_file_notes`
        `context_length` - length of each context

    Returns: a list of training pairs (x, t), with len(x) == context_length
    """
    D = []
    for i in range(len(notes) - context_length):
        seq = notes[i:i+context_length]
        next_note = notes[i+context_length]
        D.append((seq, next_note),)

    D.append((notes[-context_length:], 0),)
    # Since the note 0 never appears in any of our pieces,
    # we use note 0 to denote END OF SONG.
    return D

def make_onehot(indicies, total=128):
    """
    Convert indicies into one-hot vectors by
    first creating an identity matrix of shape [total, total],
    then indexing the appropriate columns of that identity matrix.

    Parameters:
        `indices` - a numpy array of some shape where
                    the value in these arrays should correspond to category
                    indices (e.g. note values between 0-127)
        `total` - the total number of categories (e.g. total number of notes)

    Returns: a numpy array of one-hot vectors
        If the `indices` array is shaped (N,)
           then the returned array will be shaped (N, total)
        If the `indices` array is shaped (N, D)
           then the returned array will be shaped (N, D, total)
        ... and so on.
    """
    I = np.eye(total)
    return I[indicies]

def generate_midi(notes, outfile):
    from mido import MidiFile, MidiTrack, Message

    new_mid = MidiFile()
    new_track = MidiTrack()
    new_mid.tracks.append(new_track)

    for note in notes:
        new_track.append(Message('note_on', note=note, velocity=64, time=128))
    new_mid.save(outfile)

In [35]:
print(get_midi_file_notes('/Users/peter/Downloads/nottingham-dataset-master/MIDI/melody/ashover1.mid'))

[76, 74, 71, 69, 71, 72, 71, 67, 69, 76, 74, 71, 69, 71, 72, 71, 65, 67, 76, 74, 71, 69, 71, 72, 71, 67, 69, 76, 74, 71, 69, 71, 72, 71, 65, 67, 76, 79, 76, 77, 74, 72, 69, 65, 76, 79, 76, 77, 74, 72, 69, 67, 76, 79, 76, 77, 74, 72, 69, 65, 76, 79, 76, 77, 74, 72, 69, 67]


In [36]:
sample_song = get_midi_file_notes('/Users/peter/Downloads/nottingham-dataset-master/MIDI/melody/ashover1.mid')
D_sample_song = gen_input_output(sample_song)

print(len(sample_song))
print(len(D_sample_song) + 20 - 1)

68
68


In [37]:
from Models.MLPMusicGen import MLPMusicGen
from Trainer import Trainer
from Trainer.losses import *

model = MLPMusicGen(20, [64,64], activation_type="tanh")

loss_fun = CrossEntropyLoss()

trainer = Trainer(model, optimizer= None, loss_fn=loss_fun)





Trainer initialized for classification task using CrossEntropyLoss loss.


In [42]:
seed = sample_song[:20]
notes = model.generate_piece(seed = seed, max_len = 30)

generated = generate_midi(notes, 'untrained.mid')
print(f"Generated {len(notes)} notes: {notes}") 

play_midi('untrained.mid')

Generated 30 notes: [76, 74, 71, 69, 71, 72, 71, 67, 69, 76, 74, 71, 69, 71, 72, 71, 65, 67, 76, 74, 39, 118, 35, 77, 25, 37, 2, 67, 120, 79]


error: '-F' is an illegal option at this place, only -b option is allowed here.
fluidsynth: error: fluid_is_soundfont(): fopen() failed: 'File does not exist.'
Parameter 'test.wav' not a SoundFont or MIDI file or error occurred identifying it.
error: '-r' is an illegal option at this place, only -b option is allowed here.
fluidsynth: error: fluid_is_soundfont(): fopen() failed: 'File does not exist.'
Parameter '44100' not a SoundFont or MIDI file or error occurred identifying it.
fluidsynth: warning: End of the MIDI file reached, but not all notes have received a note off event! OFFing them now! Run with --verbose to spot pending voices.


FluidSynth runtime version 2.5.1
Copyright (C) 2000-2025 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.



ValueError: rate must be specified when data is a numpy array or list of audio samples.